<a href="https://colab.research.google.com/github/goteguru/kmooc_python/blob/main/notebooks/en/kmooc_07_1_numpy_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NumPy

Earlier we mentioned that Python is an interpreted language and (by default) uses only a single CPU core, so large-scale computations can be slow. The numpy package helps with this problem — it is a highly optimized array processing library.

(By large-scale computation here we really mean large-scale: doing a few tens of thousands of operations on a few tens of thousands of numbers is not a problem for modern machines using basic Python, but inverting matrices of size tens of thousands by tens of thousands (hundreds of millions of elements) becomes problematic in an interpreted language.)

This package is very popular in data processing, numerical modeling and artificial intelligence — practically a de facto standard. So it's useful to know what it can do and how it works.


In [ ]:
# first step: import the package!
import numpy as np
# immediately renamed it to np (common convention) to type less.
# So be sure to run this cell,
# otherwise the others won't work at all.

## Data

In [ ]:
one_dimensional = np.array([1, 2, 3])
two_dimensional = np.array([[1, 2], [3, 4]])

The one-dimensional array contains three rows (even though it prints side-by-side to fit better, they are rows).

You can easily print arrays (for very large ones it prints only part) and query some useful information about them!

In [ ]:
a = np.array([[1,2,3], [4,5,6], [7,8,9]])
print("number of dimensions:", a.ndim)
print("shape:", a.shape)
print("total elements:" , a.size)
print("data type:", a.dtype)

print("the array itself:\n", a)

As you can see, an array has only one data type. This is in stark contrast to Python lists: a numpy array can only contain elements of the same type and fixed size — you cannot freely mix different types!

This is actually good, because the elements can be accessed much faster. If every value occupies exactly the same amount of space, say 8 bytes, then the n-th value is exactly 8*n bytes away from the element at index zero.

Although less common, arrays can also contain text. You just have to make sure they are of the same type. If not, NumPy (as a fallback) will try to convert them.

In [ ]:
np.array(['Alfonz', 'Csilla', 'Viktor'])

In [ ]:
np.array(['Alfonz', 12, 0.34]) # these will become three strings, even though we provided numbers

If you run the blocks above, you can see that it prints the dtype at the end. The U there means unicode data, and `<U6` means up to 6 codepoints (taking up six code units). NumPy determined this by looking at the longest element and formatting all elements to that length. NumPy array elements must have the same size, otherwise it couldn't jump quickly through the memory. So


In [ ]:
arr = np.array(["a", "kobra"])
# let's see how much space the two elements and the whole np array occupy:
len(arr[0].tobytes()), len(arr[1].tobytes()), len(arr.tobytes())

So even though the first element is a single character (4 bytes with unicode encoding and termination), because the second takes 20 bytes the first must also reserve that space (the remaining bytes are filled with zeros). In the end both occupy 20 bytes, so the total size is 40. This matters when you have millions of elements!

Moreover, since NumPy guesses the element size from the initial values (which dtype would be best), if you later try to put a larger value in, it will be truncated!

In [ ]:
arr[0] = "kerecsensólyom"
arr[0] # only the beginning fits, the rest gets truncated :-(

Therefore, if you know you may have longer strings later, it's better to specify the size manually rather than relying on the heuristic!

In [ ]:
arr = np.array(["a", "kobra"], dtype="<U200")
arr[0] = "kerecsensólyom"
arr[0] # 200 unicode codepoints is plenty!

## Array creation

Often we need arrays with a specific structure but don't want to type them by hand: array-creation functions help here:

In [ ]:
np.zeros((2, 3))      # 2x3 "all zeros" array
np.ones((3, 3))       # 3x3 ones
np.full((2,2), 7)     # every element is 7
np.eye(3)             # 3x3 identity matrix
np.arange(0, 10, 2)   # 0–10 with step 2
np.linspace(0, 1, 5)  # 5 equally spaced values in [0,1]
np.random.random((2,2)) # 2x2 random numbers
np.random.normal(size=20) # normal distribution (20 samples)
np.random.lognormal(mean=0.0, sigma=1.0, size=10) # lognormal


... or we can reshape (rearrange) an existing array:

In [ ]:
vector = np.arange(0,36) # numbers 0-35 (a single long vector)
print(vector)

matrix = np.reshape(vector, (6,6)) # the same reshaped into a 6x6 matrix.
print(matrix)

flat = np.ravel(matrix) # flatten it again
print(flat)

Note that the numbers of the "vector" were printed side-by-side to fit better, but they are actually rows. So it is a 36-row (vertical) matrix (vector).

In [ ]:
tens = np.arange(10,100,10)  # [10,20...90]
ones_seq = np.arange(1,10) # [1,2,...9]

print(np.hstack([tens, ones_seq])) # concatenate horizontally
print(np.vstack([tens, ones_seq])) # concatenate vertically


## Indexing

NumPy arrays can be indexed with square brackets just like lists, but on turbo mode! Using commas* you can specify multiple dimensions (since arrays can be 2D, 3D or higher), and you can even filter with boolean values (masking).

\* Of course, we know that the comma actually denotes a tuple, so here it's really saying that NumPy can index its data structures with tuples.

In [ ]:
m = np.arange(1,17).reshape((4,4)) # 4x4 matrix

print(m) # the whole matrix
print(m[0,0]) # top-left corner
print(m[-1,-1]) # bottom-right corner (same as [3,3], but from the end)
print(m[1,3]) # second row, fourth element (indices start at 0)
print(m[1:3]) # from second row up to the fourth (i.e. rows 2 and 3)

If you omit a number on either side of the colon (range), it means "from the start" or "to the end" just like with lists. So if you omit both sides you get all elements. This is also true for lists, but with arrays it becomes more useful because they can be multi-dimensional and you might want all elements along the first axis but only a subset along the others!

If instead of a simple index you pass a list (or tuple), you can request multiple indices at once.

In [ ]:
print(matrix[:,1]) # every row but only the 2nd column!
print("-"*30) # separator

print(matrix[[0,9]]) # only the first and the last row
print("-"*30) # separator

print(matrix[:,[0,9]]) # only the first and the last column
# print(matrix[:,(0,9)]) # <--- could have written it like this as well

Because any N-dimensional array can be considered as N+1-dimensional by inserting a new dimension of size one, NumPy allows us to insert dimensions (or "axes") anywhere. This becomes truly useful together with broadcasting rules (see below).

In [ ]:
arr = np.array([1,2,3,4])
print(arr.shape) # this is four rows.

(4,)


## Vectorized operations

NumPy's real power is that you don't need to step through arrays element-by-element with relatively slow Python code; it can perform operations on the whole array "at once".

In reality it doesn't do everything truly simultaneously, but with optimized code and by using multiple CPU cores it is significantly faster than writing explicit loops.

Run these cells one after another:

In [ ]:
a = np.array([1, 2, 3])
a

In [ ]:
a * 2         # array([2, 4, 6])


In [ ]:
a + 10        # array([11, 12, 13])

In [ ]:
a*a + 1

The regular Python math functions only work on scalars, so NumPy defines vectorized variants (sqrt, log, etc.) that work on arrays:

In [ ]:
np.sqrt(a) # this is not the sqrt from the math module

In [ ]:
np.log10(a) # base-10 logarithm

In [ ]:
np.pow(a,3) # cubes

So when you perform an operation, it applies to every element. The result does not even have to be numeric — it can be boolean, for example:

In [ ]:
a > 2

As you can see, the result array contains boolean values (True where the condition holds). At first this may not seem very useful, but it's extremely handy because NumPy arrays can be indexed with boolean arrays! In that case you get only the values where the boolean index is True.

In [ ]:
a = np.array([10,20,30])
a[[True, False, True]] # <- take the first, not the second, take the third.


array([10, 30])

In [ ]:
x = np.linspace(0,10,1000) # one thousand evenly spaced values between 0 and 10.
# let's compute some wild function for every value:
y = 2 * x*x * np.sin(2*x) + x - 1 # y = 2x² * sin(2x) + x - 1

y<0 # At which points is y negative?
y[y<0] # request all y values where y is negative.

In [ ]:
# numerical changes of y (approx):
dy = np.diff(y) # increments (999 elements)
y[1:][dy>3] # where did y increase by more than 3?


The multiplication operator (`*`) in NumPy means element-wise multiplication, not matrix multiplication. To avoid awkward calls to np.linalg.matmul, there is an operator for matrix multiplication: `@`. The transpose is also available as the .T attribute.

In [ ]:
vectors = np.array([
    [2,8],
    [9,1],
    [-2,3],
    [4,-7],
    [2,2],
])
print(vectors)
# Gram matrix (all pairwise dot-products):
vectors @ vectors.T

In [ ]:
# Compute distances from the following coordinates!
coords = np.array([
    [1, 8],
    [7, 12],
    [1, 7],
    [3, 9]
])

# d = ... <-- some computation

#### solution:

In [ ]:
x,y = coords[:,0], coords[:,1]
d = np.sqrt( x*x + y*y )

# or:
d = np.sqrt(x**2 + y**2)

# or simply:
d = np.linalg.norm(coords, axis=1)
d

## Aggregations

We can very quickly compute aggregations (sum, mean, max, etc.).



In [ ]:
a = np.array([2,9,8,7,1])
print(a.sum())
print(a.mean()) # mean
print(a.min()) # minimum element
print(a.max())
print(a.argmax())    # index of maximum element
print(a.argmin())


If the array is multi-dimensional, you can specify axes along which to aggregate:

In [ ]:
m = np.array([[1, 2, 3],
              [4, 5, 6]])

m.sum(axis=0)   # column-wise sums → [5, 7, 9]
m.sum(axis=1)   # row-wise sums → [6, 15]


array([ 6, 15])

## Broadcasting (expansion)

NumPy automatically expands ("broadcasts") smaller arrays so that operations make sense. If the two arrays have the same shape, it performs element-wise operations. If an axis is missing on the smaller array, it repeats the smaller array along that axis as many times as needed.


In [ ]:
m = np.array([[1,2,3],
              [4,5,6]])

v = np.array([10, 20, 30])

m + v

In [ ]:
# logically it actually does the same when operating with a constant:
#
a = np.array([1,2,3,4])

print(a * [3,3,3,3])
print(a * 3)

[ 3  6  9 12]
[ 3  6  9 12]


Broadcasting can be combined with axis-expansion, so we can easily compute combinations. If we intentionally insert a new axis, broadcasting will repeat elements along that axis, so we don't need to write loops!

Here's an example: we have N values and want to compute differences (all pairwise combinations). We insert a new row axis once (which can be repeated), and a new column axis once (also repeated), so when we combine them we get all pairwise differences.

In [ ]:
values = np.array([1,5,15,40])

values[None,:] - values[:,None]

Okay, actually we didn't strictly need to add the extra axis on the subtraction's first part, because they were already rows and broadcasting would have extended over the columns anyway, but this form may be easier to understand.

Try removing the `values[None,:]` index.

# Example

Let's see how easily we can solve a two-dimensional linear regression:

In [ ]:

data = np.array(
            #x1                 x2            y
       [[  75.375     ,    9.4453125 ,  125.00445389],
       [  14.7421875 ,   32.3125    ,  -65.97093686],
       [  63.96875   ,   87.375     , -132.55686565],
       [  22.875     ,   61.        , -135.55599849],
       [  70.8125    ,    4.7578125 ,  129.77077384],
       [  58.9375    ,   85.625     , -138.95654107],
       [   8.6953125 ,    4.62109375,    3.55415825],
       [  50.78125   ,   62.6875    ,  -84.62982924],
       [  42.53125   ,   19.125     ,   29.47322941],
       [  89.        ,   32.625     ,   81.7848131 ]])


In [ ]:
xs = data[:, 0:2]
y = data[:,2]

# theoretical solution
ginv = np.linalg.inv(xs.T @ xs)
w = ginv @ xs.T @ y
w # weights: ~2 and ~3


In [ ]:
# Of course, instead of linear algebra we could have used an optimization routine:

w = np.linalg.lstsq(xs, y)[0]
w

In [ ]:
# errors:
errors = y - xs @ w
print(errors)

# sum of squared errors:
np.sum(np.power(errors, 2))


## Computation speed

NumPy not only yields more concise code (thanks to broadcasting rules) but is significantly faster due to its efficient C implementation.

In [ ]:
import numpy as np
import math # for distance calculations
import time # for timing

# Let's compute lengths for two million (random) vectors
N = 2_000_000
rng = np.random.default_rng(0)
x_np = rng.random(N)
y_np = rng.random(N)

# pure Python solution:
x_py = x_np.tolist() # convert to python lists
y_py = y_np.tolist()

t0 = time.perf_counter() # measure only this part
# the fastest pure-Python option is a list comprehension:
r_py = [math.hypot(x_py[i], y_py[i]) for i in range(N)]
t1 = time.perf_counter()

# numpy solution:
t2 = time.perf_counter()
r_np = np.hypot(x_np, y_np)
t3 = time.perf_counter()

print(f"Python: {t1 - t0:.3f}s")
print(f"NumPy : {t3 - t2:.3f}s")

Note: if you run the timing cell several times, you may notice that NumPy can get even faster once it's "warmed up" (e.g. it has loaded the vectorized hypot implementation).